# ERA5 geopotentials and storm track

## Import packages

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.path as mpath
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from c3s_eqc_automatic_quality_control import diagnostics, download

## Define Parameters

In [ ]:
# Time
start = "1991"
stop = "2020"  # None: present

# Season to plot
season = "DJF"

## Define request

In [ ]:
collection_id = "reanalysis-era5-pressure-levels"
request = {
    "product_type": "reanalysis",
    "format": "grib",
    "variable": "geopotential",
    "pressure_level": "500",
    "area": [90, -180, 30, 180],
    "time": ["00:00", "12:00"],
}
requests = download.update_request_date(request, start=start, stop=stop)

## Functions to cache

In [ ]:
def seasonal_bandpass(ds, weights_mapper, reductions):
    if weights_mapper:
        ds = diagnostics.rolling_weighted_filter(ds, weights_mapper)
    datasets = []
    for reduction in reductions:
        func = getattr(diagnostics, f"seasonal_weighted_{reduction}")
        datasets.append(func(ds, weights=False).expand_dims(reduction=[reduction]))
    return xr.concat(datasets, "reduction")

## Define bandpass weights (Blackmon, JAS, 1976)

In [ ]:
blackmon_weights = [
    -0.00304,
    -0.00013,
    -0.00967,
    -0.01917,
    -0.002,
    0.03043,
    0.03281,
    0.00411,
    0.00335,
    0.04193,
    0.0283,
    -0.09233,
    -0.19477,
    -0.10201,
    0.14335,
    0.27769,
]
blackmon_weights.extend(blackmon_weights[:-1][::-1])

## Download and transform

In [ ]:
datasets = []
for weights in (None, blackmon_weights):
    ds = download.download_and_transform(
        collection_id,
        requests,
        transform_func=seasonal_bandpass,
        transform_func_kwargs={
            "weights_mapper": {"forecast_reference_time": weights} if weights else {},
            "reductions": ("mean", "std"),
        },
        chunks={"year": 1, "time": 1},
        transform_chunks=False,
        concat_dim="forecast_reference_time",
        combine="nested",
    )
    datasets.append(ds.expand_dims(bandpass=[True if weights else False]))
da = xr.concat(datasets, "bandpass")["z"].sel(season=season)
with xr.set_options(keep_attrs=True):
    da /= 9.8

## Plot

In [ ]:
# Circle used to set the bounday
theta = np.linspace(0, 2 * np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)

fig, axes = plt.subplots(
    *(1, 3),
    figsize=(15, 7.5),
    subplot_kw={"projection": ccrs.NorthPolarStereo(central_longitude=0)},
)
axes = iter(axes.flatten())
for bandpass, da_bandpass in da.groupby("bandpass"):
    for reduction, da_reduction in da_bandpass.groupby("reduction"):
        if reduction == "mean" and not bandpass:
            levels_list = [range(5000, 6000, 50), range(5000, 6000, 200)]
        elif reduction == "std" and not bandpass:
            levels_list = [range(0, 200, 10), range(0, 200, 40)]
        elif reduction == "std" and bandpass:
            levels_list = [range(0, 100, 4), [30, 60]]
        else:
            continue

        ax = next(axes)
        for levels in levels_list:
            cs = ax.contour(
                da_reduction["longitude"],
                da_reduction["latitude"],
                da_reduction,
                levels=levels,
                colors="k",
                linewidths=0.5,
                linestyles="solid",
                transform=ccrs.PlateCarree(),
            )
        ax.clabel(cs)
        ax.coastlines(color="darkgray")
        ax.add_feature(cfeature.LAND)
        ax.set_boundary(circle, transform=ax.transAxes)
        ax.set_title(
            f"{'Band-pass ' if bandpass else ''}{reduction.title()}", fontsize=16
        )